
<h1 id="Programming-Language-Trend-and-Sentiment-Analysis">Programming Language Trend and Sentiment Analysis<a class="anchor-link" href="#Programming-Language-Trend-and-Sentiment-Analysis">¶</a></h1><p><strong>Project by:</strong></p>
<p><strong>Satyajeet Maharana,
Hitesh Patel,
Mike Akiki,
Shreya Pandey</strong></p>
<p><strong>Team 8</strong></p>



<p>We plan to analyze stackoverflow data to figure out</p>
<p>Trends in programming languages. 
Analysis of comment will help understanding overall satisfaction of different programmers. 
To find language in use according to region.</p>



<h1 id="Data-Collection-from-stackoverflow-archive">Data Collection from stackoverflow archive<a class="anchor-link" href="#Data-Collection-from-stackoverflow-archive">¶</a></h1><p>We collected below data of different sizes from stack overflow archive link : <a href="https://archive.org/download/stackexchange">https://archive.org/download/stackexchange</a></p>
<ol>
<li>stackoverflow.votes.7z</li>
<li>stackoverflow-tags.7z</li>
<li>stackoverflow-users.7z</li>
<li>stackoverflow-comments.7z</li>
<li>stackoverflow-posts.7z</li>
</ol>



<h1 id="ER-Diagram">ER Diagram<a class="anchor-link" href="#ER-Diagram">¶</a></h1>



<p>Click here for the PDF file of the ER Diagram: <a href="https://drive.google.com/file/d/1FmlUIAijHdBy09rG-wf6zZ5WaP5u5hTm/view?usp=sharing">ER Diagram PDF</a></p>



<p><img alt="alt text" src="https://i.ibb.co/WGncwHr/ER-Diagram.jpg"/></p>



<h1 id="Data-Cleaning-(converting-xml-file-into-csv)-using-python">Data Cleaning (converting xml file into csv) using python<a class="anchor-link" href="#Data-Cleaning-(converting-xml-file-into-csv)-using-python">¶</a></h1><p>The datafiles we received were in sizes of about 10GB - 60 GB in XML format. After we ran the below code to ignore the columns we do not need,further clean the data, and  convert the file format to CSV.</p>
<h2 id="The-below-code-was-run-on-the-Bigdata-server-with-files-present-in-the-Bigdata-local-folders.">The below code was run on the Bigdata server with files present in the Bigdata local folders.<a class="anchor-link" href="#The-below-code-was-run-on-the-Bigdata-server-with-files-present-in-the-Bigdata-local-folders.">¶</a></h2><p>After Cleaning our data size in csv was within range 100 MBs to 5GB</p>
<p>Below is the code:</p>


In [ ]:

#! /usr/bin/python3

import xml.etree.ElementTree as etree
import csv
 
with open('stackoverflow_comments.csv', 'w+') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Id','PostId','Score','Text','CreationDate','UserId'])
    i = 0
    for event, elem in etree.iterparse('/bigtemp/DealingF18GB/DealingF18GB8/Project Data/Comments.xml', events=('start','start-ns')):
        print(i)
        if(elem.attrib.get('Id')!= None):
            Id = elem.attrib.get('Id')
            if(Id != None):
                Id = Id.encode('utf-8')
            PostId = elem.attrib.get('PostId')
            if(PostId != None):
                PostId = PostId.encode('utf-8')
            Score = elem.attrib.get('Score')
            if(Score != None):
              Score = Score.encode('utf-8')
            Text = elem.attrib.get('Text')
            if(Text != None):
                Text = Text.encode('utf-8')
            CreationDate = elem.attrib.get('CreationDate')
            if(CreationDate!=None):
                CreationDate = CreationDate.split('T')[0].encode('utf-8')
            UserId = elem.attrib.get('UserId')
            if(UserId != None):
                UserId = UserId.encode('utf-8')
            
            filewriter.writerow([Id,PostId,Score,Text,CreationDate,UserId])
            elem.clear()
            i = i+1




<p><strong>Note :</strong> The above code was run in Bigdata server and is not expected to run successfully in Notebooks.</p>



<h1 id="Data-Load-in-Hive">Data Load in Hive<a class="anchor-link" href="#Data-Load-in-Hive">¶</a></h1><p>We uploaded data into hive and executed Hive Queries to aggregate and get count within valid range with respect to different parameters like region,creationdate,tags,etc</p>
<h2 id="The-below-code-was-was-run-in-Bigdata-server-directly.-The-required-files-were-uploaded-to-Bigdata-server-using-SFTP-(Cyberduck-software)">The below code was was run in Bigdata server directly. The required files were uploaded to Bigdata server using SFTP (Cyberduck software)<a class="anchor-link" href="#The-below-code-was-was-run-in-Bigdata-server-directly.-The-required-files-were-uploaded-to-Bigdata-server-using-SFTP-(Cyberduck-software)">¶</a></h2><p>Below are the sample hive Queries:</p>


In [ ]:

create external table Stackoverflow_posts (id  int, tags string, owneruserid int, lastedate string, answercount int, creationdate string, acceptedanswerid int)
row format delimited fields terminated by ',' 
location '/user/DealingF18GB8/Project/Hivetables/â€™;
tblproperties ("skip.header.line.count"="1");

create external table stackoverflow_posts  (id int, tags string, owneruserid int, lastdate string, answercount int, creationdate string,acceptedanserid int) row format delimited fields terminated by ','
location '/user/DealingF18GB8/Project/Hivetables_stackposts/' ;
create external table stackoverflow_comments  (id int, postid int, score int, text string, creationdate string,userid int) row format delimited fields terminated by ','
location '/user/DealingF18GB8/Project/Hivetables_stackcomments/' ;
create external table stackoverflow_users  (Id int,Reputation int,CreationDate string,LastAccessDate string,Location string,Views int,UpVotes int,DownVotes int,AccountId int) row format delimited fields terminated by ','
location '/user/DealingF18GB8/Project/Hivetables_stackusers' ;

create external table Stackoverflow_tags  (id int, tag string) ;
insert into table stackoverflow_tags SELECT Id, tag FROM stackoverflow_posts lateral view explode(split(tags,'\;')) tags AS tag;


create table joined_tag_date(id int,tag string ,creationdate string) row format delimited fields terminated by ',';
insert into table joined_tag_date select t.id ,t.tag ,p.creationdate from stackoverflow_posts p inner join stackoverflow_tags t on p.id=t.id 


create table aggregate_tag_date(tag string ,creationdate string,count int) row format delimited fields terminated by ',';
insert into table aggregate_tag_date select tag,creationdate,count(*) from joined_tag_date WHERE tag IS NOT NULL AND creationdate IS NOT NULL group by tag,creationdate;
insert overwrite local directory '/bigtemp/DealingF18GB/aggregate_tag_date.csv' row format delimited fields terminated by ',' select * from aggregate_tag_date;




<p><strong>Note :</strong> The above code was run in Bigdata server and is not expected to run successfully in Notebooks.</p>



<h1 id="Sentiment-Analysis-Using-NLTK-on-comments:">Sentiment Analysis Using NLTK on comments:<a class="anchor-link" href="#Sentiment-Analysis-Using-NLTK-on-comments:">¶</a></h1><p>We calculated individual sentiment of every comment in comments.csv file contaings millions of rows.</p>
<h2 id="The-below-code-was-run-on-our-local-machines.-The-comments.csv-file-was-downloaded-from-the-Bigdata-server-and-a-new-CSV-file-containing-the-sentiment-of-the-comments-were-generated-using-the-below-code.">The below code was run on our local machines. The comments.csv file was downloaded from the Bigdata server and a new CSV file containing the sentiment of the comments were generated using the below code.<a class="anchor-link" href="#The-below-code-was-run-on-our-local-machines.-The-comments.csv-file-was-downloaded-from-the-Bigdata-server-and-a-new-CSV-file-containing-the-sentiment-of-the-comments-were-generated-using-the-below-code.">¶</a></h2><p>The code is as below:</p>


In [ ]:

import nltk



In [ ]:

#!/usr/bin/env python 

#import nltk

import sys
import re
import textblob
#from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from textblob import TextBlob 

def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False
for line in sys.stdin:

	if(re.search(r'^$',line)):
		continue
	else:
		line = line.decode('utf-8')
		line = line.split(',')
		if(len(line)>3):
			if(RepresentsInt(line[0]) and RepresentsInt(line[1])):
				analysis = TextBlob(line[3])

				# set sentiment 
				if analysis.sentiment.polarity > 0: 
					print ('{},{},{}\n').format(line[0].encode('utf-8'),line[1].encode('utf-8'),1),
				elif analysis.sentiment.polarity == 0: 
					print ('{},{},{}\n').format(line[0].encode('utf-8'),line[1].encode('utf-8'),0),
				else: 
					print ('{},{},{}\n').format(line[0].encode('utf-8'),line[1].encode('utf-8'),-1),




<p><strong>Note : </strong>The above code was run in Bigdata server and is not expected to run successfully in Notebooks.</p>



<p>As of now, all the data was loaded in Hive tables. For easy analysis of our data in Python, we first ran the aggregation jobs in Hive. The resulting CSV files were then loaded to SQL tables.</p>
<p>There were lot of steps involved in actually migrating the data from Hive to SQL. We followed the basic steps as :</p>
<h2 id="Load-Hive-Data-into-a-CSV-on-HDFS-&gt;&gt;-Copy-the-CSV-file-from-HDFS-to-Local-&gt;&gt;-Upload-the-CSV-file-into-SQL-tables.">Load Hive Data into a CSV on HDFS &gt;&gt; Copy the CSV file from HDFS to Local &gt;&gt; Upload the CSV file into SQL tables.<a class="anchor-link" href="#Load-Hive-Data-into-a-CSV-on-HDFS-&gt;&gt;-Copy-the-CSV-file-from-HDFS-to-Local-&gt;&gt;-Upload-the-CSV-file-into-SQL-tables.">¶</a></h2><p>Time being a constraint, we applied the above steps. However, there could be a smarter way of doing it. Which we plan on exploring in future projects.</p>
<p>The below SQL tables were generated :</p>
<ol>
<li>Project_Tag_Date : This table stores the aggregate count of the posts data, which was grouped by Tags and Date fields. The resultant table has the below structure : </li>
</ol>
<ul>
<li>Tag (varchar)</li>
<li>CreationDate (date)</li>
<li>Count (int)</li>
</ul>
<ol>
<li>Project_Tag_Location : This table stores the aggregate count of the posts data, which was grouped by Tags and Location fields. The resultant table has the below structure :</li>
</ol>
<ul>
<li>Tag (varchar)</li>
<li>Location (varchar)</li>
<li>Count (int)</li>
</ul>
<ol>
<li>Project_Tag_Sentiment : This table stores the aggregate sentiment of the posts data, which was grouped by the Tags field. The resultant table has the below structure :</li>
</ol>
<ul>
<li>Tag (varchar)</li>
<li>Sentiment Score (int)</li>
</ul>



<h1 id="Data-cleaning-COMPLETE!!-Now-for-the-FUN-stuff---Analysis-in-Python">Data cleaning COMPLETE!! Now for the FUN stuff - Analysis in Python<a class="anchor-link" href="#Data-cleaning-COMPLETE!!-Now-for-the-FUN-stuff---Analysis-in-Python">¶</a></h1>



<h2 id="Step-1:-Installation-of-the-pre-requisite-packages">Step 1: Installation of the pre-requisite packages<a class="anchor-link" href="#Step-1:-Installation-of-the-pre-requisite-packages">¶</a></h2><p><strong>Now starts the code which we have run on our Python Notebooks. </strong></p>


In [ ]:

!sudo pip3 install -q -U sql_magic

!sudo apt-get update

!sudo apt-get install python-mysqldb

!sudo apt-get install python3-dev libmysqlclient-dev

!sudo pip3 install mysqlclient




<h2 id="Step-2:-Preparation-of-the-SQL-engine-for-accessing-MySQL-server-data">Step 2: Preparation of the SQL engine for accessing MySQL server data<a class="anchor-link" href="#Step-2:-Preparation-of-the-SQL-engine-for-accessing-MySQL-server-data">¶</a></h2>


In [ ]:

from sqlalchemy import create_engine
conn_string = 'mysql://{user}:{password}@{host}/?charset=utf8'.format(
    host = 'bigdata.stern.nyu.edu', 
    user = 'DealingF18GB8',
    password = 'DealingF18GB8!!',
    encoding = 'utf-8')
engine = create_engine(conn_string)



In [ ]:

%reload_ext sql_magic
%config SQL.conn_name = 'engine'



In [ ]:

%%read_sql
use DealingF18GB8;



In [ ]:

%%read_sql
show tables ;



In [ ]:

%%read_sql 
describe DealingF18GB8.Project_Tag_Location;



In [ ]:

%%read_sql 
describe DealingF18GB8.Project_Tag_Date;




<h2 id="Step-3:-Importing-Pandas">Step 3: Importing Pandas<a class="anchor-link" href="#Step-3:-Importing-Pandas">¶</a></h2>


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter




<h2 id="Step-4:-Query-aggregate-data-from-the-table">Step 4: Query aggregate data from the table<a class="anchor-link" href="#Step-4:-Query-aggregate-data-from-the-table">¶</a></h2>


In [ ]:

%%read_sql
show tables ;




<h1 id="Code-for-Geotagging-the-locations">Code for Geotagging the locations<a class="anchor-link" href="#Code-for-Geotagging-the-locations">¶</a></h1><h1 id="Why-Geotagging-is-important-in-our-project?">Why Geotagging is important in our project?<a class="anchor-link" href="#Why-Geotagging-is-important-in-our-project?">¶</a></h1><p>As of now our data is in the table Project_Tag_Location table. However, the values present in the location field for the respective tags are not in uniform format. For eg, one row contains "New York" and another row contains "United States". This is not good.</p>
<p>Hence, as an important step, we did a geotagging of our data. We queried the aggregate data from "Project_Tag_Location" table and grouped by location. You may notice that the limit has been set to 2400. The reason for putting a limit in our results is that we found the first 2400 records based on frequency to be of relevance.</p>
<h1 id="How-we-did-Geotagging?">How we did Geotagging?<a class="anchor-link" href="#How-we-did-Geotagging?">¶</a></h1><p>We used Google Map APIs for Geotagging our data. The Map API is not free of cost, but we receive $ 300 credit in the free trial period. We made the best utilization of that. :-)</p>
<h1 id="Next-step">Next step<a class="anchor-link" href="#Next-step">¶</a></h1><p>1st Step : After we did the Geotagging, it was important that the data be immediately stored in a resultant table, in case we faced any real time issues. We went ahead and stored the resultant data in a new table called "Project_Tag_FreqByCity".</p>
<p>2nd Step : Now we have data distrubuted by Cities. We further aggregate the data in the "Project_Tag_FreqByCity" to another table called "Project_Tag_FreqByCountry".</p>
<h2 id="Phew-!!-Lot-of-work-but-we-finally-have-a-precise-table-with-the-relevant-aggregated-data.-Stay-tuned-for-more.">Phew !! Lot of work but we finally have a precise table with the relevant aggregated data. Stay tuned for more.<a class="anchor-link" href="#Phew-!!-Lot-of-work-but-we-finally-have-a-precise-table-with-the-relevant-aggregated-data.-Stay-tuned-for-more.">¶</a></h2>


In [ ]:

%%read_sql df_location
select location,SUM(count) freq from Project_Tag_Location WHERE tag IS NOT NULL AND tag != '' AND location != '' GROUP BY location ORDER BY freq DESC LIMIT 2400;



In [ ]:

import requests
GOOGLE_MAPS_API_URL = 'https://maps.googleapis.com/maps/api/geocode/json'


address = 'Bangalore'
params = {
    #'key':'AIzaSyC2L937vXMH3QpJIPHTs1uuH1H3dLsuH', #removed key due to request limits
    'address': address,
    'sensor': 'false',
    'lang': 'en'
}

# Do the request and get the response data
req = requests.get(GOOGLE_MAPS_API_URL, params=params)
res = req.json()
res



In [ ]:

import requests
GOOGLE_MAPS_API_URL = 'https://maps.googleapis.com/maps/api/geocode/json'

df_location_only = pd.DataFrame()
for index, row in df_location.iterrows():
    try:
      df_location.loc[index,'location'] = row['location'].replace('|','')
      address = row['location']
      params = {
          #'key':'AIzaSyC2L937vXMH3QpJIPHTs1uuH1H3dLsuH',  #removed key due to request limits
          'address': address,
          'sensor': 'false',
          'lang': 'en'
      }

      # Do the request and get the response data
      req = requests.get(GOOGLE_MAPS_API_URL, params=params)
      res = req.json()

      results = res['results']
      listaddresscomp = results[0]['address_components']
      location = results[0]['geometry']['location']
      lat = location['lat']
      lng = location['lng']

      df_location.loc[index,'lat'] = lat
      df_location.loc[index,'lng'] = lng

      for addcomp in listaddresscomp: 
        longname = addcomp['long_name']
        shortname = addcomp['short_name']
        typename = addcomp['types'][0]

        if(typename == 'country'):
          countryname = longname
          df_location.loc[index,'countryname'] = countryname

      df_location_only.loc[index,'rawloc'] = row['location']
      df_location_only.loc[index,'lat'] = row['lat']
      df_location_only.loc[index,'lng'] = row['lng']
      df_location_only.loc[index,'countryname'] = row['countryname']
      df_location_only.loc[index,'freq'] = row['freq']
    except Exception as e: 
      print(e)



In [ ]:

df_location_only.fillna(value="",inplace=True)



In [ ]:

import MySQLdb as mdb

con = mdb.connect(host = 'bigdata.stern.nyu.edu', 
                  user = 'DealingF18GB8',                  
                  passwd = 'DealingF18GB8!!', 
                  charset='utf8', use_unicode=True);


# Query to create a database
db_name = 'DealingF18GB8'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
cursor = con.cursor()
cursor.execute(create_db_query)
cursor.close()


cursor = con.cursor()
table_name = 'Project_Locations'
# Create a table
# Drop it if it exists
create_drop_locations='''drop table if exists {db}.{table}'''.format(db=db_name,table=table_name)

cursor.execute(create_drop_locations)
# The {db} and {table} are placeholders for the parameters in the format(....) statement
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table}(rawloc varchar(255),lat varchar(255),lng varchar(255),countryname varchar(255),freq int)'''.format(db=db_name, table=table_name)
cursor.execute(create_table_query)
cursor.close()


query_template = '''INSERT INTO {db}.{table}(rawloc,lat,lng,countryname,freq) VALUES (%s, %s, %s, %s, %s)'''.format(db=db_name, table=table_name)
cursor = con.cursor()

rawloc = ''
formattedadd  = '' 
lat  = '' 
lng  = ''
countryname  = ''
freq = ''

for index, row in df_location_only.iterrows():
    rawloc = row['rawloc']
    lat  = row['lat'] 
    lng  = row['lng'] 
    countryname  = row['countryname']
    freq = row['freq']

    
    if(rawloc == None):
      rawloc = ''
    if(lat == None):
      lat = ''
    if(lng == None):
      lng = ''
    if(countryname == None):
      countryname =''
    if(freq == None):
      freq = 0
    query_parameters = (rawloc, lat, lng, countryname,freq)
    cursor.execute(query_template, query_parameters)

con.commit()
cursor.close()




<h2 id="Step-5:-Now-plotting-our-Geotagged-data-in-a-world-map-to-identify-Developers-from-Different-countries.">Step 5: Now plotting our Geotagged data in a world map to identify Developers from Different countries.<a class="anchor-link" href="#Step-5:-Now-plotting-our-Geotagged-data-in-a-world-map-to-identify-Developers-from-Different-countries.">¶</a></h2>


In [ ]:

!!sudo pip3 install -U geopy



In [ ]:

import MySQLdb as mdb

con = mdb.connect(host = 'bigdata.stern.nyu.edu', 
                  user = 'DealingF18GB8',                  
                  passwd = 'DealingF18GB8!!', 
                  charset='utf8', use_unicode=True);


# use a database
db_name = 'DealingF18GB8'

#cursor = con.cursor()
table_name = 'Project_Tag_FreqByCity'

;

query_country_freq=''' SELECT SUM(freq) as fr,countryname FROM {db}.{table} WHERE countryname != '' AND freq != 0 GROUP BY countryname ORDER BY fr DESC '''.format(db=db_name,table=table_name)

df_country_freq = pd.read_sql(query_country_freq, con)

df_country_freq.head(20)


from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="nameofapp")

for index,row in df_country_freq.iterrows():
    location = geolocator.geocode(row['countryname'])
    df_country_freq.loc[index,'lat'] = location.latitude
    df_country_freq.loc[index,'lng'] = location.longitude
df_country_freq



In [ ]:

!sudo pip3 install -U folium
!sudo pip3 install -U geopandas




<h1 id="Final-Outcomes">Final Outcomes<a class="anchor-link" href="#Final-Outcomes">¶</a></h1>



<h2 id='1.-From-the-Tag-data-present-in-"Project_Tag_Date"-table,-we-find-the-most-commonly-used-languages,-databases-and-frameworks.-The-data-is-then-plotted-in-a-bar-chart-for-easier-understanding.'>1. From the Tag data present in "Project_Tag_Date" table, we find the most commonly used languages, databases and frameworks. The data is then plotted in a bar chart for easier understanding.<a class="anchor-link" href='#1.-From-the-Tag-data-present-in-"Project_Tag_Date"-table,-we-find-the-most-commonly-used-languages,-databases-and-frameworks.-The-data-is-then-plotted-in-a-bar-chart-for-easier-understanding.'>¶</a></h2>


In [ ]:

import MySQLdb as mdb

con = mdb.connect(host = 'bigdata.stern.nyu.edu', 
                  user = 'DealingF18GB8',                  
                  passwd = 'DealingF18GB8!!', 
                  charset='utf8', use_unicode=True);


# use a database
db_name = 'DealingF18GB8'
#use_db_query = "use {db}".format(db=db_name)
#cursor = con.cursor()
#cursor.execute(use_db_query)
#cursor.close()


#cursor = con.cursor()
table_name = 'Project_Tag_Date'

query_tag_freq='''SELECT tag Community,count Number_Of_Posts FROM {db}.{table} WHERE tag NOT LIKE '20%' '''.format(db=db_name,table=table_name)

df_tag_freq = pd.read_sql(query_tag_freq, con)

df_tag_freq.head(20)


df_tag_by_freq = df_tag_freq.groupby('Community')['Number_Of_Posts'].sum().sort_values().tail(15)


df_tag_by_freq.plot(kind='barh',figsize=(20,20),title="Top Developer Communities")




<h2 id="2.-From-top-100-languages,-we-find-below-:">2. From top 100 languages, we find below :<a class="anchor-link" href="#2.-From-top-100-languages,-we-find-below-:">¶</a></h2><p>A.   The most satisfied developer communities</p>
<p>B.   The most frustrated developer communities</p>



<h2 id="A.-The-most-satisfied-developer-communities">A. The most satisfied developer communities<a class="anchor-link" href="#A.-The-most-satisfied-developer-communities">¶</a></h2>


In [ ]:

import MySQLdb as mdb

con = mdb.connect(host = 'bigdata.stern.nyu.edu', 
                  user = 'DealingF18GB8',                  
                  passwd = 'DealingF18GB8!!', 
                  charset='utf8', use_unicode=True);

db_name = 'DealingF18GB8'
table_name = 'Project_Tag_Date'
query_tag_freq_ordered='''SELECT tag,SUM(count) freq FROM {db}.{table} WHERE tag != '' AND tag NOT LIKE '20%' GROUP BY tag ORDER BY freq DESC LIMIT 200'''.format(db=db_name,table=table_name)

df_tag_freq_ordered = pd.read_sql(query_tag_freq_ordered, con)



  #SELECT tag,SUM(sentiment) FROM DealingF18GB8.Project_Tag_Sentiment WHERE tag != '' AND tag NOT LIKE '20%' GROUP BY tag;

  


  # use a database

  #use_db_query = "use {db}".format(db=db_name)
  #cursor = con.cursor()
  #cursor.execute(use_db_query)
  #cursor.close()

query_tag_sentiment = {}
df_tag_sentiment = {}
  #cursor = con.cursor()
for index,row in df_tag_freq_ordered.iterrows():
    table_name = 'Project_Tag_Sentiment'
    key = row["tag"]
    query_tag_sentiment ='''SELECT tag Community,sentiment totalsentiment FROM {db}.{table} WHERE tag != '' AND tag NOT LIKE '20%' AND tag = '{tagval}' '''.format(db=db_name,table=table_name,tagval=key)

    df_tag_sentiment[key] = pd.read_sql(query_tag_sentiment, con)
df_list = []

df_communities = pd.concat(df_tag_sentiment.values())

df_communities['Satisfaction(%)'] = 100*df_communities['totalsentiment']/sum(df_communities['totalsentiment'])



df_happiest_communities = df_communities.sort_values('Satisfaction(%)',ascending=False)
df_happiest_communities = df_happiest_communities.iloc[:15]
ax = df_happiest_communities.plot.barh(x='Community', y='Satisfaction(%)',figsize=(20,20),title='Happiest Developer Communities')




<h2 id="B.-The-most-frustrated-developer-communities">B. The most frustrated developer communities<a class="anchor-link" href="#B.-The-most-frustrated-developer-communities">¶</a></h2>


In [ ]:

import MySQLdb as mdb

con = mdb.connect(host = 'bigdata.stern.nyu.edu', 
                  user = 'DealingF18GB8',                  
                  passwd = 'DealingF18GB8!!', 
                  charset='utf8', use_unicode=True);

db_name = 'DealingF18GB8'
table_name = 'Project_Tag_Date'
query_tag_freq_ordered='''SELECT tag,SUM(count) freq FROM {db}.{table} WHERE tag != '' AND tag NOT LIKE '20%' GROUP BY tag ORDER BY freq DESC LIMIT 200'''.format(db=db_name,table=table_name)

df_tag_freq_ordered = pd.read_sql(query_tag_freq_ordered, con)



  #SELECT tag,SUM(sentiment) FROM DealingF18GB8.Project_Tag_Sentiment WHERE tag != '' AND tag NOT LIKE '20%' GROUP BY tag;

  


  # use a database

  #use_db_query = "use {db}".format(db=db_name)
  #cursor = con.cursor()
  #cursor.execute(use_db_query)
  #cursor.close()

query_tag_sentiment = {}
df_tag_sentiment = {}
  #cursor = con.cursor()
for index,row in df_tag_freq_ordered.iterrows():
    table_name = 'Project_Tag_Sentiment'
    key = row["tag"]
    query_tag_sentiment ='''SELECT tag Community,sentiment totalsentiment FROM {db}.{table} WHERE tag != '' AND tag NOT LIKE '20%' AND tag = '{tagval}' '''.format(db=db_name,table=table_name,tagval=key)

    df_tag_sentiment[key] = pd.read_sql(query_tag_sentiment, con)
df_list = []

df_communities = pd.concat(df_tag_sentiment.values())

df_communities['Satisfaction(%)'] = 100*df_communities['totalsentiment']/sum(df_communities['totalsentiment'])


df_frustrated_communities = df_communities.sort_values('Satisfaction(%)',ascending=True)
df_frustrated_communities = df_frustrated_communities.iloc[:15]
ax = df_frustrated_communities.plot.barh(x='Community', y='Satisfaction(%)',figsize=(20,20),title='Frustrated Developer Communities')




<h1 id="3.-Here's-the-plot-on-the-world-map.-Check-the-countries-with-highest-number-of-developers">3. Here's the plot on the world map. Check the countries with highest number of developers<a class="anchor-link" href="#3.-Here's-the-plot-on-the-world-map.-Check-the-countries-with-highest-number-of-developers">¶</a></h1><p>This is an interactive map provided by folium. We can zoom in and click on the circles to see the name of the countries.</p>
<p>As of today, we can see that the countries with the highest number of developers are in USA, India and UK.</p>


In [ ]:

import folium
fmap = folium.Map(location=[20,0], zoom_start=2, tiles="Mapbox Bright")
fmap

totalfreq = sum(df_country_freq["fr"])
for name, row in df_country_freq.iterrows():
    # Define the opacity of the marker to be proportional to the percentage of bikes in the station
    opacity = 0.5
    # Make the color green for the working stations, red otherwise
    color = "cyan"
    # The size of the marker is proportional to the number of docks
    size = row["fr"]/500000

    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["lat"], row["lng"]],radius = size,fill=True,fill_opacity = opacity,color=color,fill_color = color,popup=row["countryname"]+"("+ str(int(row["fr"]))+" posts)").add_to(fmap)
fmap




<h1 id="Conclusions-:">Conclusions :<a class="anchor-link" href="#Conclusions-:">¶</a></h1>



<p>We observed that most popular programming language is javascript and Java, whereas most satisfied developer communities are javascript and java programmer. 
Hence the might conclude that since javscript and java developer are most satisfied,  causing the language to be more popular.
Beside the most frustrated developer community is elasticeserch and reactjs users.</p>
<p>Also maximum contributions are from USA,India,UK and Germany region.</p>



<h1 id="Lessons-Learned-:">Lessons Learned :<a class="anchor-link" href="#Lessons-Learned-:">¶</a></h1>



<p>We understood the difficulties faced during data cleaning phase, mostly time required to process the huge amount of data. The Hadoop framework helps in substantially decreasing these time in real-world. Further, even after data cleaning, we faced discrepancies in a data file and we had to ignore those entries in our analysis. We did write python program for data cleaning, which is fast, efficient and easy to write. We did sentiment analysis on comments to figure out overall satisfaction of different programmer communities. After aggregating data on Hive QL, we imported it into SQL tables and data frames. This helped in understanding the whole process of getting a piece of valuable information from raw data and using it to get better insights on correlations between different unrelated variables after the visualization step.</p>



<h1 id="Thank-you">Thank you<a class="anchor-link" href="#Thank-you">¶</a></h1>
